# Classifier

meer grid based dingen toevoegen, nu gaan we niet hoger komen


In [14]:
from SimpleCV import *
from IPython.display import HTML
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import sklearn 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import cv2
import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import linear_model




In [2]:
#df = pd.read_csv("../dataset-numpy/dataset_analysis.csv")
# wordt niet hoger door normalizen
df = pd.read_csv("../dataset-numpy/dataset_analysis_normalized.csv")
df.head()


# split df in data (X) and labels (y)
X, y = df.iloc[:,:-1], df.iloc[:,-1]

# create train and test data and labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.20, random_state=42)
df.describe()

blob_area  blob_centroid_x  blob_centroid_y   blob_angle  \
count  1920.000000      1920.000000      1920.000000  1920.000000   
mean      0.386221         0.469217         0.467661     0.779948   
std       0.187824         0.139196         0.182193     0.368169   
min       0.000000         0.000000         0.000000     0.000000   
25%       0.253482         0.378048         0.344453     0.863493   
50%       0.341226         0.462067         0.459344     0.973220   
75%       0.481894         0.552328         0.562653     1.000000   
max       1.000000         1.000000         1.000000     1.000000   

        blob_width  number_of_holes  centroid_is_empty  centroid_top_is_empty  \
count  1920.000000      1920.000000        1920.000000            1920.000000   
mean      0.522865         0.188542           0.295312               0.211458   
std       0.101772         0.287165           0.456302               0.408449   
min       0.000000         0.000000           0.000000               0.000000   
25%       0.500000         0.000000           0.000000               0.000000   
50%       0.500000         0.000000           0.000000               0.000000   
75%       0.600000         0.500000           1.000000               0.000000   
max       1.000000         1.000000           1.000000               1.000000   

       centroid_bottom_is_empty     grid_2_0     ...          grid_4_7  \
count               1920.000000  1920.000000     ...       1920.000000   
mean                   0.348958     0.393815     ...          0.781217   
std                    0.476765     0.333569     ...          0.331414   
min                    0.000000     0.000000     ...          0.000000   
25%                    0.000000     0.062500     ...          0.625000   
50%                    0.000000     0.375000     ...          1.000000   
75%                    1.000000     0.687500     ...          1.000000   
max                    1.000000     1.000000     ...          1.000000   

          grid_5_0     grid_5_1     grid_5_2     grid_5_3     grid_5_4  \
count  1920.000000  1920.000000  1920.000000  1920.000000  1920.000000   
mean      0.379785     0.533919     0.516829     0.510319     0.586458   
std       0.383374     0.376778     0.400050     0.382840     0.370961   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.125000     0.000000     0.125000     0.250000   
50%       0.250000     0.562500     0.562500     0.500000     0.687500   
75%       0.750000     0.937500     0.937500     0.875000     0.937500   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

          grid_5_5     grid_5_6     grid_5_7        label  
count  1920.000000  1920.000000  1920.000000  1920.000000  
mean      0.551400     0.591960     0.472656     4.532813  
std       0.369417     0.375565     0.403487     2.868122  
min       0.000000     0.000000     0.000000     0.000000  
25%       0.187500     0.250000     0.000000     2.000000  
50%       0.625000     0.750000     0.437500     5.000000  
75%       0.875000     0.937500     0.875000     7.000000  
max       1.000000     1.000000     1.000000     9.000000  

[8 rows x 42 columns]

In [3]:
# min_max_scaler = preprocessing.MinMaxScaler() # Scale the training features
# training_set = min_max_scaler.fit_transform(X_train)
# # Scale the test features (using the same settings as the trainingset)
# test_set = min_max_scaler.transform(X_test)
# print test_set

# PCA

In [4]:
from sklearn.decomposition import PCA

pca = PCA(.95)
pca.fit(X_train)
pca.n_components_
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

# Naive Bayes

In [5]:
#Naive Bayes Gaussian
gnb = GaussianNB()
gnb.fit(X_train, y_train)
#y_pred = gnb.predict(X_test)
print "Gaussian :",gnb.score(X_test, y_test)

#Naive Bayes Bernoulli
bnb = BernoulliNB()
bnb.fit(X_train, y_train)
#y_pred = gnb.predict(X_test)
print "Bernoulli:",bnb.score(X_test, y_test)


Gaussian : 0.921875
Bernoulli: 0.770833333333


# Random forest randomize search

In [30]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

#number of trees 
n_estimators=[int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)] 

#number of features considered for splitting at leaf node
max_features =['sqrt', 'auto']

#method for sampling data points (with or without replacement)
bootstrap=[True, False]

#min number of data points allowed in a leaf node
min_samples_leaf=[1,2,4]

#min number of data points placed in a node before the node is split
min_samples_split=[2,5,10]

#max number of levels in each decision tree
max_depth=[int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

randomgrid = {'n_estimators':n_estimators,
             'max_features': max_features,
             'bootstrap': bootstrap,
             'min_samples_leaf': min_samples_leaf,
             'min_samples_split': min_samples_split,
             'max_depth': max_depth}

rf = RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = randomgrid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
rf_random.best_params_
rf_random.best_score_

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=auto, max_depth=30 
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=auto, max_depth=30 
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=400, min_samples_split=5, max_features=auto, max_depth=30 
[CV] bootstrap=True, min_samples_leaf=1, n_estimators=2000, min_samples_split=5, max_features=auto, max_depth=10 


KeyboardInterrupt: 

In [ ]:
rf_random.best_params_

# Random forest Grid search

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [30,40,50,60,70],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1,2,4],
    'min_samples_split': [2,5,10],
    'n_estimators': [800,900,1000,1100,1200,1300,1400]
}

# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_
grid_search.best_score_

In [ ]:
grid_search.best_params_


In [92]:
#Random Forest
# Create a random forest classifier
clfRF = RandomForestClassifier(n_jobs=2, random_state=0)
# Train the classifier to take the training features and learn how they relate to the training y
clfRF.fit(X_train, y_train)
print "Random forest:",clfRF.score(X_test, y_test)

Random forest: 0.9140625


In [93]:
#randomize search outcome Random forest: 0.966145833333
# clfRF = RandomForestClassifier(bootstrap= False,  max_depth= 50, max_features= 'sqrt',
#                                min_samples_leaf = 1, min_samples_split = 2, n_estimators = 1000)
#grid search outcome:  0.96875
clfRF = RandomForestClassifier(bootstrap= False,  max_depth= 40, max_features= 'sqrt', min_samples_leaf = 1, min_samples_split = 2, n_estimators = 900)

clfRF.fit(X_train, y_train)
print "Random forest:",clfRF.score(X_test, y_test)
print(clfRF.get_params())

Random forest: 0.947916666667
{'warm_start': False, 'oob_score': False, 'n_jobs': 1, 'min_impurity_decrease': 0.0, 'verbose': 0, 'max_leaf_nodes': None, 'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 900, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'criterion': 'gini', 'random_state': None, 'min_impurity_split': None, 'max_features': 'sqrt', 'max_depth': 40, 'class_weight': None}


# Decision tree

In [94]:
from sklearn.tree import DecisionTreeClassifier
#Decision tree  0.8671875
clfDT = DecisionTreeClassifier(min_samples_leaf = 1, min_samples_split = 2, max_features = None, max_depth = 52)
clfDT.fit(X_train, y_train) 
print "Decision tree:",clfDT.score(X_test, y_test)

Decision tree: 0.8203125


# KNN with grid search

In [95]:
#making the instance
clfKNN = KNeighborsClassifier(n_jobs=-1)
#Hyper Parameters Set
params = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10],
          'leaf_size':[1,2,3,5],
          'weights':['uniform', 'distance'],
          'algorithm':['auto', 'ball_tree','kd_tree','brute'],
          'n_jobs':[-1]}
#Making models with hyper parameters sets
clfKNN1 = GridSearchCV(clfKNN, param_grid=params, n_jobs=-1, verbose=1)
#Learning
clfKNN1.fit(X_train,y_train)
#The best hyper parameters set
print("Best parameters:\n",clfKNN1.best_params_)
#Prediction
prediction=clfKNN1.predict(X_test)
#importing the metrics module
from sklearn import metrics
#evaluation(Accuracy)
print("Accuracy:",metrics.accuracy_score(prediction,y_test))
#evaluation(Confusion Metrix)
print(metrics.confusion_matrix(prediction,y_test))

Fitting 3 folds for each of 320 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   29.3s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.1min
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


('Best parameters:\n', {'n_neighbors': 5, 'n_jobs': -1, 'weights': 'distance', 'leaf_size': 1, 'algorithm': 'auto'})
('Accuracy:', 0.96875)
[[31  0  0  0  0  0  0  0  0  0]
 [ 0 47  0  0  0  0  0  0  2  0]
 [ 0  0 42  0  0  0  0  0  0  0]
 [ 0  0  0 41  0  1  0  0  0  0]
 [ 0  0  0  0 28  0  0  0  1  1]
 [ 0  0  0  1  0 31  0  0  1  0]
 [ 0  0  0  0  0  0 39  0  0  0]
 [ 0  0  0  0  0  0  0 44  0  2]
 [ 0  0  0  0  0  0  0  0 36  0]
 [ 0  0  0  0  3  0  0  0  0 33]]


[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:  1.9min finished


In [96]:
# KNN
# niet accurate, hoeveelheid neighbors:
# Score KNN:  1 0.971354166667
# Score KNN:  2 0.9609375
# Score KNN:  3 0.963541666667
# Score KNN:  4 0.966145833333
# Score KNN:  5 0.963541666667
# Score KNN:  6 0.963541666667
# Score KNN:  7 0.963541666667
# Score KNN:  8 0.9609375
# Score KNN:  9 0.963541666667



clfKNN = KNeighborsClassifier(n_neighbors = 4, n_jobs = -1, weights= 'distance', leaf_size= 1, algorithm= 'auto')
# for i in range (1,10):
#     # Create a classifier with k 
#     clfKNN = KNeighborsClassifier(n_neighbors=i)
#     # Train the classifier with the training data and training labels
#     clfKNN.fit(X_train, y_train)

#     # Score the classifier
#     # (calculates the mean accuracy of the given test data) 
#     score = clfKNN.score(test_set, y_test)
#     print "Score KNN: ", i , score
clfKNN.fit(X_train, y_train)

# Score the classifier
# (calculates the mean accuracy of the given test data) 
score = clfKNN.score(X_test, y_test)
print score
# print "Score KNN: ", i , score

0.973958333333


# SGD

In [97]:
from sklearn import linear_model
from sklearn.model_selection import ParameterGrid

clf = linear_model.SGDClassifier()
clf.get_params()
# params = { 'loss': ['log'],
#         'penalty': ['elasticnet'],
#         'alpha': [10 ** x for x in range(-6, 1)],
#     'l1_ratio': [0, 0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 1],
#           'n_jobs':[-1]}
                            
# clf1 = GridSearchCV(clf, param_grid=params, n_jobs=-1, verbose=1)
# #Learning
# clf1.fit(training_set,y_train)
# #The best hyper parameters set
# print("Best Hyper Parameters:\n",clf1.best_params_)
# #Prediction
# prediction=clf1.predict(test_set)
# #importing the metrics module
# from sklearn import metrics
# #evaluation(Accuracy)
# print("Accuracy:",metrics.accuracy_score(prediction,y_test))
# #evaluation(Confusion Metrix)
# print("Confusion Metrix:\n",metrics.confusion_matrix(prediction,y_test))

{'alpha': 0.0001,
 'average': False,
 'class_weight': None,
 'epsilon': 0.1,
 'eta0': 0.0,
 'fit_intercept': True,
 'l1_ratio': 0.15,
 'learning_rate': 'optimal',
 'loss': 'hinge',
 'max_iter': None,
 'n_iter': None,
 'n_jobs': 1,
 'penalty': 'l2',
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'tol': None,
 'verbose': 0,
 'warm_start': False}

# SVM with Gridsearch

In [7]:
# {'kernel': 'rbf', 'C': 1.9000000000000001, 'gamma': 0.161}
# 0.978515625

# {'kernel': 'rbf', 'C': 1.9000000000000001, 'gamma': 0.19600000000000001}
# 0.977213541667

parameters = {'kernel' : ('linear', 'rbf'), 'C': np.arange(0.1, 2, 0.1), 
              'gamma': np.arange(.001, .2, 0.005)}


clf = GridSearchCV(svm.SVC(), parameters, verbose=1, n_jobs=-1, cv = 5)
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

print clf.best_params_
print clf.best_score_

Fitting 5 folds for each of 1520 candidates, totalling 7600 fits


[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 388 tasks      | elapsed:   34.4s
[Parallel(n_jobs=-1)]: Done 888 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1588 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 2488 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 3588 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 4888 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 6388 tasks      | elapsed:  5.1min


{'kernel': 'rbf', 'C': 1.9000000000000001, 'gamma': 0.19600000000000001}
0.977213541667


[Parallel(n_jobs=-1)]: Done 7600 out of 7600 | elapsed:  5.9min finished


In [6]:
# 0.979166666667 met pca en normalized zonder contours

clf = svm.SVC(kernel ="rbf", C=1.9, gamma=0.161)
clf.fit(X_train, y_train)

print clf.score(X_test, y_test)

0.979166666667


In [41]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

predictions = clf.predict(X_test)
labels = y_test

print confusion_matrix (labels, predictions)
print classification_report(labels, predictions)

# for p, l in zip(predictions, labels):
#     print p, l

[[31  0  0  0  0  0  0  0  0  0]
 [ 0 47  0  0  0  0  0  0  0  0]
 [ 0  0 42  0  0  0  0  0  0  0]
 [ 0  0  0 40  0  2  0  0  0  0]
 [ 0  0  0  0 28  0  0  0  0  3]
 [ 0  0  0  0  0 32  0  0  0  0]
 [ 0  0  0  0  0  0 39  0  0  0]
 [ 0  0  0  0  0  0  0 44  0  0]
 [ 0  0  0  0  0  1  0  0 39  0]
 [ 0  0  0  0  1  0  0  1  0 34]]
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        31
          1       1.00      1.00      1.00        47
          2       1.00      1.00      1.00        42
          3       1.00      0.95      0.98        42
          4       0.97      0.90      0.93        31
          5       0.91      1.00      0.96        32
          6       1.00      1.00      1.00        39
          7       0.98      1.00      0.99        44
          8       1.00      0.97      0.99        40
          9       0.92      0.94      0.93        36

avg / total       0.98      0.98      0.98       384



In [12]:
from sklearn.metrics import accuracy_score

accuracy_score(labels, predictions)

0.98177083333333337

# Logistic Regression

In [11]:
pipeline = Pipeline([('logreg',LogisticRegression(multi_class="multinomial",solver="lbfgs"))])
parameters = {'logreg__C':np.arange(0.01,100,10)}
clfLG = GridSearchCV(pipeline,parameters,cv=5, n_jobs=-1, verbose =2)
clfLG.fit(X_train, y_train)
print("Best parameters:\n",clfLG.best_params_)
print ("Score:\n", clfLG.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] logreg__C=0.01 ..................................................
[CV] logreg__C=0.01 ..................................................
[CV] ................................... logreg__C=0.01, total=   0.1s
[CV] ................................... logreg__C=0.01, total=   0.1s
[CV] logreg__C=0.01 ..................................................
[CV] logreg__C=0.01 ..................................................
[CV] ................................... logreg__C=0.01, total=   0.0s
[CV] logreg__C=0.01 ..................................................
[CV] ................................... logreg__C=0.01, total=   0.1s
[CV] logreg__C=10.01 .................................................
[CV] ................................... logreg__C=0.01, total=   0.1s
[CV] logreg__C=10.01 .................................................
[CV] .................................. logreg__C=10.01, total=   0.2s
[CV] logreg__C=1

KeyboardInterrupt: 

In [12]:
clfLG = LogisticRegression(random_state=0, solver='lbfgs',  multi_class='multinomial', C =10.01 )
clfLG.fit(X_train, y_train)
print clfLG.score(X_test, y_test)
#0.947916666667

0.947916666667


# SGD

In [15]:
clf = linear_model.SGDClassifier(loss = 'log', n_iter =1000)
clf.fit(X_train, y_train)
print clf.score(X_test, y_test)

#default = 0.934895833333
#n_iter =1000 = 0.919270833333
#loss = 'log', n_iter =1000 = 0.934895833333

/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.934895833333


# Export best model

In [ ]:
from sklearn.externals import joblib

clf = svm.SVC(kernel= 'rbf', C= 1.9, gamma= 0.161) 
clf.fit(X_train, y_train)
# Save the classifier in a file
joblib.dump(clf, 'svcModel.joblib')

In [262]:
# from sklearn.pipeline import Pipeline

# # {'pca__n_components': 24, 'svm__C': 2.1, 'svm__kernel': 'rbf', 'svm__gamma': 0.20000000000000004}
# # 0.977213541667

# pipeline = Pipeline([('pca', PCA()), ('svm', svm.SVC())])
# parameter_grid = {
# #     'pca__n_components' : range(1,24),
#     'svm__kernel' : ['linear', 'rbf'],
#     'svm__C' : np.arange(1, 5, 0.1),
#     'svm__gamma': np.arange(.1, 1, .1)
# }
# # parameters = {'kernel' : ('linear', 'rbf'), 'C': np.arange(1,100), 'gamma': np.arange(.1, 1, 0.05)}

# # svc = svm.SVC(gamma = j)
# # print j
# # n_jobs-1 = aantal cores/multithreading
# # verbose print spul
# clf = GridSearchCV(pipeline, parameter_grid, verbose=1, n_jobs=-1, cv = 5)
# clf.fit(X_train, y_train)

# print clf.best_params_
# print clf.best_score_

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 388 tasks      | elapsed:   34.0s
[Parallel(n_jobs=-1)]: Done 888 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1588 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 2488 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 3588 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 3600 out of 3600 | elapsed:  5.5min finished


{'svm__C': 1.4000000000000004, 'svm__kernel': 'rbf', 'svm__gamma': 0.20000000000000001}
0.977864583333


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    8.5s


{'kernel': 'linear', 'C': 1, 'gamma': 0}
0.963541666667


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    9.4s finished
